In [1]:
from vanna.openai import OpenAI_Chat
from vanna.vannadb import VannaDB_VectorStore
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

class MyVanna(VannaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = 'sakila'
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key='vn-15d6c72d5f9e41c097f399820309cba8',config=config)
        OpenAI_Chat.__init__(self, config=config)

# vn = MyVanna(config={'api_key': os.getenv('OPENAI_API_KEY'), 'model': 'gpt-4o-mini'})

vn = MyVanna(config={
    'api_key': os.getenv('OPENAI_API_KEY'), 
    'model': 'gpt-4o-mini',
    'email': 'elielecas00@gmail.com'
})
# vn.remove_training_data(id=1)



# Connexion à SQLite
sqlite_db_path = 'sakila.db'  # Chemin vers votre fichier SQLite
vn.connect_to_sqlite(sqlite_db_path)


In [2]:
response = vn.run_sql("SELECT name FROM sqlite_master WHERE type='table'")
print(response)  # Afficher la réponse brute
print(type(response))  # Pour voir si c'est bien un objet contenant du JSON



             name
0           actor
1         country
2            city
3         address
4        language
5        category
6        customer
7            film
8      film_actor
9   film_category
10      film_text
11      inventory
12          staff
13          store
14        payment
15         rental
<class 'pandas.core.frame.DataFrame'>


In [3]:
reponse = vn.ask("liste moi les 10 dernieres commandes ?")
print(reponse)

'NoneType' object has no attribute 'questions'
(None, None, None)


In [4]:
# Créer un DataFrame avec la structure attendue par get_training_plan_generic
tables = vn.run_sql("SELECT name FROM sqlite_master WHERE type='table'")

# Créer un DataFrame avec les colonnes attendues
schema_data = []
for table_name in tables['name']:
    # Obtenir les colonnes de chaque table
    columns_info = vn.run_sql(f"PRAGMA table_info({table_name})")
    
    for _, row in columns_info.iterrows():
        schema_data.append({
            'TABLE_CATALOG': 'sakila',
            'TABLE_SCHEMA': 'main',
            'TABLE_NAME': table_name,
            'COLUMN_NAME': row['name'],
            'DATA_TYPE': row['type'],
            'IS_NULLABLE': 'YES' if row['notnull'] == 0 else 'NO'
        })

df_information_schema = pd.DataFrame(schema_data)


# Générer le plan d'entraînement
plan = vn.get_training_plan_generic(df_information_schema)
print(plan)

# Entraîner le modèle
vn.train(plan=plan)


Train on Information Schema: sakila.main actor
Train on Information Schema: sakila.main country
Train on Information Schema: sakila.main city
Train on Information Schema: sakila.main address
Train on Information Schema: sakila.main language
Train on Information Schema: sakila.main category
Train on Information Schema: sakila.main customer
Train on Information Schema: sakila.main film
Train on Information Schema: sakila.main film_actor
Train on Information Schema: sakila.main film_category
Train on Information Schema: sakila.main film_text
Train on Information Schema: sakila.main inventory
Train on Information Schema: sakila.main staff
Train on Information Schema: sakila.main store
Train on Information Schema: sakila.main payment
Train on Information Schema: sakila.main rental


Exception: ('Error adding documentation', {'error': {'code': -32600, 'message': 'Invalid Request: No email'}, 'jsonrpc': '2.0'})

In [ ]:
# Training data pour la base Sakila

# 1. Films et acteurs
vn.train(
    question="Quels sont les 10 films les plus loués ?",
    sql="""
    SELECT 
        f.title AS [Titre du film],
        COUNT(r.rental_id) AS [Nombre de locations]
    FROM film f
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    GROUP BY f.film_id, f.title
    ORDER BY COUNT(r.rental_id) DESC
    LIMIT 10
    """
)

# 2. Clients
vn.train(
    question="Quels sont les 5 clients qui ont dépensé le plus d'argent ?",
    sql="""
    SELECT 
        c.first_name || ' ' || c.last_name AS [Nom complet],
        SUM(p.amount) AS [Total dépensé]
    FROM customer c
    JOIN payment p ON c.customer_id = p.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
    ORDER BY SUM(p.amount) DESC
    LIMIT 5
    """
)

# 3. Catégories de films
vn.train(
    question="Quelle est la catégorie de films la plus populaire ?",
    sql="""
    SELECT 
        cat.name AS [Catégorie],
        COUNT(r.rental_id) AS [Nombre de locations]
    FROM category cat
    JOIN film_category fc ON cat.category_id = fc.category_id
    JOIN film f ON fc.film_id = f.film_id
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
    GROUP BY cat.category_id, cat.name
    ORDER BY COUNT(r.rental_id) DESC
    LIMIT 1
    """
)

# 4. Acteurs populaires
vn.train(
    question="Quels sont les acteurs qui ont joué dans le plus de films ?",
    sql="""
    SELECT 
        a.first_name || ' ' || a.last_name AS [Nom de l'acteur],
        COUNT(fa.film_id) AS [Nombre de films]
    FROM actor a
    JOIN film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY a.actor_id, a.first_name, a.last_name
    ORDER BY COUNT(fa.film_id) DESC
    LIMIT 10
    """
)

# 5. Revenus par magasin
vn.train(
    question="Quel magasin génère le plus de revenus ?",
    sql="""
    SELECT 
        s.store_id AS [ID Magasin],
        SUM(p.amount) AS [Revenus totaux]
    FROM store s
    JOIN staff st ON s.store_id = st.store_id
    JOIN payment p ON st.staff_id = p.staff_id
    GROUP BY s.store_id
    ORDER BY SUM(p.amount) DESC
    """
)

# 6. Films par langue
vn.train(
    question="Combien de films sont disponibles en français ?",
    sql="""
    SELECT 
        l.name AS [Langue],
        COUNT(f.film_id) AS [Nombre de films]
    FROM language l
    JOIN film f ON l.language_id = f.language_id
    WHERE l.name = 'French'
    GROUP BY l.language_id, l.name
    """
)

# 7. Durée moyenne des films
vn.train(
    question="Quelle est la durée moyenne des films par catégorie ?",
    sql="""
    SELECT 
        cat.name AS [Catégorie],
        AVG(f.length) AS [Durée moyenne (minutes)]
    FROM category cat
    JOIN film_category fc ON cat.category_id = fc.category_id
    JOIN film f ON fc.film_id = f.film_id
    GROUP BY cat.category_id, cat.name
    ORDER BY AVG(f.length) DESC
    """
)

# 8. Locations récentes
vn.train(
    question="Quelles sont les 10 dernières locations effectuées ?",
    sql="""
    SELECT 
        f.title AS [Film],
        c.first_name || ' ' || c.last_name AS [Client],
        r.rental_date AS [Date de location]
    FROM rental r
    JOIN inventory i ON r.inventory_id = i.inventory_id
    JOIN film f ON i.film_id = f.film_id
    JOIN customer c ON r.customer_id = c.customer_id
    ORDER BY r.rental_date DESC
    LIMIT 10
    """
)

# 9. Films non retournés
vn.train(
    question="Combien de films sont actuellement en retard de retour ?",
    sql="""
    SELECT COUNT(*) AS [Films en retard]
    FROM rental r
    WHERE r.return_date IS NULL 
    AND r.rental_date < date('now', '-7 days')
    """
)

# 10. Revenus mensuels
vn.train(
    question="Quels sont les revenus totaux par mois ?",
    sql="""
    SELECT 
        strftime('%Y-%m', p.payment_date) AS [Mois],
        SUM(p.amount) AS [Revenus totaux]
    FROM payment p
    GROUP BY strftime('%Y-%m', p.payment_date)
    ORDER BY [Mois] DESC
    """
)

In [ ]:
training_data = vn.get_training_data()
training_data

In [ ]:
reponse = vn.ask("liste moi les 10 dernieres commandes ?")
print(reponse)

In [ ]:
reponse = vn.ask("liste moi les 10 dernieres commandes ?")
print(reponse)

In [ ]:
from vanna.flask import VannaFlaskApp
VannaFlaskApp(vn, allow_llm_to_see_data=True).run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
